# 🫀 Heart Segmentation Advanced - Setup & Configuration

<a href="https://colab.research.google.com/github/leonardobora/pratica-aprendizado-de-maquina/blob/main/Heart_Segmentation_Advanced/00_Setup_and_Configuration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 📋 Objetivos deste Notebook

Este notebook configura o ambiente completo para o projeto de segmentação cardíaca avançada:

- ⚙️ **Configuração do ambiente** (Colab/Local)
- 📦 **Instalação de dependências**
- 🔧 **Configuração de paths e constantes**
- 🎯 **Verificação de hardware**
- 🌱 **Configuração de seeds para reprodutibilidade**
- 📊 **Setup de logging e visualização**

---

In [18]:
# =============================================================================
# 🔍 VERIFICAÇÃO DE AMBIENTE
# =============================================================================

import sys
import os

# Verificar se estamos no Google Colab
try:
    import google.colab
    IN_COLAB = True
    print("🌐 Executando no Google Colab")
except ImportError:
    IN_COLAB = False
    print("🖥️ Executando em ambiente local")

# Informações do sistema
print(f"🐍 Python version: {sys.version}")
print(f"📁 Current directory: {os.getcwd()}")

🖥️ Executando em ambiente local
🐍 Python version: 3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]
📁 Current directory: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced


In [2]:
# =============================================================================
# 📦 INSTALAÇÃO DE DEPENDÊNCIAS
# =============================================================================

# Instalar dependências específicas que podem não estar disponíveis
dependencies = [
    'nibabel',           # Para arquivos NIfTI
    'SimpleITK',         # Processamento de imagens médicas
    'scikit-image',      # Operações morfológicas
    'seaborn',          # Visualização avançada
    'plotly',           # Visualização interativa
    'tqdm',             # Barras de progresso
]

if IN_COLAB:
    for package in dependencies:
        print(f"📥 Instalando {package}...")
        %pip install -q {package}
else:
    print("💻 Em ambiente local - certifique-se de que as dependências estão instaladas")
    print(f"Dependências necessárias: {', '.join(dependencies)}")

💻 Em ambiente local - certifique-se de que as dependências estão instaladas
Dependências necessárias: nibabel, SimpleITK, scikit-image, seaborn, plotly, tqdm


In [3]:
# =============================================================================
# 🔗 CONFIGURAÇÃO DO GOOGLE DRIVE (SE NO COLAB)
# =============================================================================

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    print("✅ Google Drive montado com sucesso!")
    
    # Definir path base no Drive
    DRIVE_BASE_PATH = '/content/drive/MyDrive'
    PROJECT_PATH = os.path.join(DRIVE_BASE_PATH, 'Heart_Segmentation_Advanced')
    DATASET_PATH = os.path.join(DRIVE_BASE_PATH, 'Medical_Decathlon/Task02_Heart')
    
    # Criar diretórios do projeto se não existirem
    os.makedirs(PROJECT_PATH, exist_ok=True)
    os.makedirs(os.path.join(PROJECT_PATH, 'models'), exist_ok=True)
    os.makedirs(os.path.join(PROJECT_PATH, 'outputs'), exist_ok=True)
    
    print(f"📁 Projeto configurado em: {PROJECT_PATH}")
    print(f"📊 Dataset esperado em: {DATASET_PATH}")
else:
    # Configuração para ambiente local
    PROJECT_PATH = os.getcwd()
    DATASET_PATH = os.path.join(PROJECT_PATH, 'data', 'Task02_Heart')
    print(f"📁 Projeto local em: {PROJECT_PATH}")
    print(f"📊 Dataset esperado em: {DATASET_PATH}")

📁 Projeto local em: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced
📊 Dataset esperado em: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\data\Task02_Heart


In [8]:
# =============================================================================
# 📚 IMPORTS PRINCIPAIS
# =============================================================================

# Imports padrão
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import logging
from datetime import datetime
import json
import pickle
from pathlib import Path
from typing import Tuple, List, Dict, Any, Optional, Union

# Deep Learning
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision import transforms

# Processamento de imagens
import cv2
from skimage import transform, morphology, measure, filters
from skimage.segmentation import clear_border
import nibabel as nib
import SimpleITK as sitk

# Machine Learning
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix

# Visualização
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Utilitários
from tqdm.auto import tqdm
import glob

print("✅ Todos os imports realizados com sucesso!")

✅ Todos os imports realizados com sucesso!


In [7]:
# Instalação de dependências necessárias
import subprocess
import sys

def install_package(package):
    """Instala um pacote usando pip"""
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado com sucesso")
    except subprocess.CalledProcessError:
        print(f"❌ Erro ao instalar {package}")

# Lista de pacotes necessários
required_packages = [
    "torch",
    "torchvision", 
    "numpy",
    "matplotlib",
    "seaborn",
    "opencv-python",
    "scikit-image",
    "scipy",
    "tqdm",
    "plotly",
    "pandas",
    "nibabel",      # Para arquivos NIfTI médicos
    "SimpleITK",    # Processamento de imagens médicas
    "scikit-learn"  # Machine learning
]

print("📦 Instalando dependências necessárias...")
for package in required_packages:
    try:
        __import__(package.replace("-", "_"))
        print(f"✅ {package} já está instalado")
    except ImportError:
        print(f"🔄 Instalando {package}...")
        install_package(package)

print("🎉 Instalação de dependências concluída!")

📦 Instalando dependências necessárias...
✅ torch já está instalado
✅ torchvision já está instalado
✅ numpy já está instalado
✅ matplotlib já está instalado
✅ seaborn já está instalado
🔄 Instalando opencv-python...
✅ opencv-python instalado com sucesso
🔄 Instalando scikit-image...
✅ scikit-image instalado com sucesso
✅ scipy já está instalado
✅ tqdm já está instalado
✅ plotly já está instalado
✅ pandas já está instalado
🔄 Instalando nibabel...
✅ nibabel instalado com sucesso
🔄 Instalando SimpleITK...
✅ SimpleITK instalado com sucesso
🔄 Instalando scikit-learn...
✅ scikit-learn instalado com sucesso
🎉 Instalação de dependências concluída!


In [10]:
# =============================================================================
# 🎯 VERIFICAÇÃO DE HARDWARE
# =============================================================================

def check_hardware():
    """Verifica hardware disponível e configurações de GPU"""
    print("🔧 VERIFICAÇÃO DE HARDWARE")
    print("=" * 50)
    
    # Informações gerais
    print(f"PyTorch version: {torch.__version__}")
    print(f"CUDA disponível: {torch.cuda.is_available()}")
    
    # Verificar GPU
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        print(f"🚀 GPU(s) disponível(is): {gpu_count}")
        for i in range(gpu_count):
            gpu_name = torch.cuda.get_device_name(i)
            gpu_memory = torch.cuda.get_device_properties(i).total_memory / 1024**3
            print(f"  GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")
        
        # Configurar GPU principal
        device = torch.device('cuda:0')
        torch.cuda.set_device(0)
        print(f"✅ GPU configurada como dispositivo padrão")
        
        # Verificar mixed precision
        try:
            # PyTorch tem suporte nativo para mixed precision
            print("✅ Mixed Precision (AMP) disponível")
        except:
            print("⚠️ Mixed Precision não disponível")
    else:
        print("⚠️ Nenhuma GPU detectada. Usando CPU.")
        device = torch.device('cpu')
    
    # Informações de memória
    import psutil
    ram_gb = psutil.virtual_memory().total / 1024**3
    print(f"💾 RAM total: {ram_gb:.1f} GB")
    
    print("=" * 50)

check_hardware()

🔧 VERIFICAÇÃO DE HARDWARE
PyTorch version: 2.7.0+cpu
CUDA disponível: False
⚠️ Nenhuma GPU detectada. Usando CPU.
💾 RAM total: 31.9 GB


In [12]:
# =============================================================================
# 🌱 CONFIGURAÇÃO DE REPRODUTIBILIDADE
# =============================================================================

def set_seeds(seed=42):
    """Define seeds para reprodutibilidade"""
    import random
    import numpy as np
    
    # Python random
    random.seed(seed)
    
    # NumPy
    np.random.seed(seed)
    
    # PyTorch
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # Para garantir reprodutibilidade completa no PyTorch
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Variável de ambiente
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    print(f"🌱 Seeds configuradas para {seed}")

# Aplicar configuração de seeds
RANDOM_SEED = 42
set_seeds(RANDOM_SEED)

# Configurar warnings
warnings.filterwarnings('ignore', category=FutureWarning)
warnings.filterwarnings('ignore', category=UserWarning)

# Configurar matplotlib
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Configuração de reprodutibilidade aplicada!")

🌱 Seeds configuradas para 42
✅ Configuração de reprodutibilidade aplicada!


In [13]:
# =============================================================================
# 📊 CONFIGURAÇÃO DE VISUALIZAÇÃO
# =============================================================================

# Configurar matplotlib
plt.style.use('default')
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3

# Configurar seaborn
sns.set_style("whitegrid")
sns.set_palette("husl")

# Configurar plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

print("🎨 Configurações de visualização aplicadas")

🎨 Configurações de visualização aplicadas


In [14]:
# =============================================================================
# 📝 CONFIGURAÇÃO DE LOGGING
# =============================================================================

def setup_logging(log_level=logging.INFO):
    """Configura sistema de logging"""
    
    # Criar diretório de logs
    log_dir = os.path.join(PROJECT_PATH, 'logs')
    os.makedirs(log_dir, exist_ok=True)
    
    # Nome do arquivo de log com timestamp
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    log_file = os.path.join(log_dir, f'heart_segmentation_{timestamp}.log')
    
    # Configurar logging
    logging.basicConfig(
        level=log_level,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(sys.stdout)
        ]
    )
    
    logger = logging.getLogger(__name__)
    logger.info("🚀 Sistema de logging iniciado")
    logger.info(f"📁 Log salvo em: {log_file}")
    
    return logger

# Inicializar logging
logger = setup_logging()

2025-06-18 11:23:51,153 - INFO - 🚀 Sistema de logging iniciado
2025-06-18 11:23:51,167 - INFO - 📁 Log salvo em: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\logs\heart_segmentation_20250618_112351.log


--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\leonardo.costa\AppData\Local\Programs\Python\Python312\Lib\logging\__init__.py", line 1163, in emit
    stream.write(msg + self.terminator)
  File "c:\Users\leonardo.costa\AppData\Local\Programs\Python\Python312\Lib\encodings\cp1252.py", line 19, in encode
    return codecs.charmap_encode(input,self.errors,encoding_table)[0]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
UnicodeEncodeError: 'charmap' codec can't encode character '\U0001f680' in position 33: character maps to <undefined>
Call stack:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\leonardo.costa\AppData\Local\Programs\Python\Python312\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\leonardo.costa\AppData\Local\Programs\Python\Python312\Lib\site-packages\traitlets\config\application.py", line 107

In [15]:
# =============================================================================
# 🔧 CONSTANTES E CONFIGURAÇÕES GLOBAIS
# =============================================================================

class Config:
    """Classe de configuração centralizada"""
    
    # Paths principais
    PROJECT_PATH = PROJECT_PATH
    DATASET_PATH = DATASET_PATH
    MODELS_PATH = os.path.join(PROJECT_PATH, 'models')
    OUTPUTS_PATH = os.path.join(PROJECT_PATH, 'outputs')
    
    # Configurações de dados
    IMAGE_SIZE = (128, 128)
    NUM_CLASSES = 3  # background, ventrículo esquerdo, miocárdio
    CLASS_NAMES = ['Background', 'Left Ventricle', 'Myocardium']
    CLASS_COLORS = ['black', 'red', 'green']
    
    # Configurações de treinamento
    BATCH_SIZE = 8
    INITIAL_EPOCHS = 100
    LEARNING_RATE = 1e-3
    VALIDATION_SPLIT = 0.2
    TEST_SPLIT = 0.1
    
    # Configurações de modelo
    BACKBONE_OPTIONS = ['resnet50', 'efficientnet-b0', 'densenet121']
    DEFAULT_BACKBONE = 'resnet50'
    
    # Configurações de augmentação
    AUGMENTATION_PARAMS = {
        'rotation_range': 15,
        'translation_range': 0.1,
        'scaling_range': (0.9, 1.1),
        'shear_range': 5,
        'flip_probability': 0.5,
        'noise_std': 0.01,
        'blur_sigma': (0.5, 1.0),
        'contrast_range': (0.8, 1.2),
        'brightness_range': 0.1
    }
    
    # Configurações de loss
    LOSS_WEIGHTS = {
        'dice': 0.4,
        'focal': 0.4,
        'boundary': 0.2
    }
    
    # Configurações de callbacks
    EARLY_STOPPING_PATIENCE = 15
    REDUCE_LR_PATIENCE = 7
    REDUCE_LR_FACTOR = 0.5
    MIN_LR = 1e-6
    
    # Configurações de pós-processamento
    MIN_COMPONENT_SIZE = 100
    MORPHOLOGY_KERNEL_SIZE = 3
    
    # Seed global
    RANDOM_SEED = RANDOM_SEED

# Criar instância de configuração
config = Config()

print("⚙️ Configurações globais carregadas:")
print(f"   📊 Dataset: {config.DATASET_PATH}")
print(f"   💾 Models: {config.MODELS_PATH}")
print(f"   📤 Outputs: {config.OUTPUTS_PATH}")
print(f"   🖼️ Image size: {config.IMAGE_SIZE}")
print(f"   🏷️ Classes: {config.NUM_CLASSES} ({', '.join(config.CLASS_NAMES)})")

⚙️ Configurações globais carregadas:
   📊 Dataset: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\data\Task02_Heart
   💾 Models: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\models
   📤 Outputs: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\outputs
   🖼️ Image size: (128, 128)
   🏷️ Classes: 3 (Background, Left Ventricle, Myocardium)


In [16]:
# =============================================================================
# ✅ VERIFICAÇÃO DE INTEGRIDADE DO SETUP
# =============================================================================

def verify_setup():
    """Verifica se o setup foi realizado corretamente"""
    
    print("🔍 VERIFICAÇÃO DE INTEGRIDADE DO SETUP")
    print("=" * 50)
    
    checks = []
    
    # Verificar paths
    paths_to_check = [
        (config.PROJECT_PATH, "Diretório do projeto"),
        (config.MODELS_PATH, "Diretório de modelos"),
        (config.OUTPUTS_PATH, "Diretório de outputs"),
    ]
    
    for path, description in paths_to_check:
        if os.path.exists(path):
            checks.append(f"✅ {description}: {path}")
        else:
            checks.append(f"❌ {description}: {path} (NÃO ENCONTRADO)")
    
    # Verificar dataset (opcional por enquanto)
    if os.path.exists(config.DATASET_PATH):
        checks.append(f"✅ Dataset: {config.DATASET_PATH}")
        
        # Verificar subdiretórios do dataset
        images_dir = os.path.join(config.DATASET_PATH, 'imagesTr')
        labels_dir = os.path.join(config.DATASET_PATH, 'labelsTr')
        
        if os.path.exists(images_dir):
            image_files = len([f for f in os.listdir(images_dir) if f.endswith('.nii.gz')])
            checks.append(f"✅ Imagens encontradas: {image_files}")
        else:
            checks.append(f"⚠️ Diretório de imagens não encontrado: {images_dir}")
            
        if os.path.exists(labels_dir):
            label_files = len([f for f in os.listdir(labels_dir) if f.endswith('.nii.gz')])
            checks.append(f"✅ Labels encontradas: {label_files}")
        else:
            checks.append(f"⚠️ Diretório de labels não encontrado: {labels_dir}")
    else:
        checks.append(f"⚠️ Dataset não encontrado: {config.DATASET_PATH}")
        checks.append("   (Certifique-se de que o dataset está no local correto)")
    
    # Verificar PyTorch e GPU
    if torch.cuda.is_available():
        checks.append("✅ GPU disponível e configurada")
    else:
        checks.append("⚠️ GPU não disponível - usando CPU")
    
    # Verificar seeds
    checks.append(f"✅ Seeds configurados: {config.RANDOM_SEED}")
    
    # Imprimir resultados
    for check in checks:
        print(check)
    
    print("=" * 50)
    
    # Log do setup
    logger.info("Setup verification completed")
    logger.info(f"Project path: {config.PROJECT_PATH}")
    logger.info(f"Dataset path: {config.DATASET_PATH}")
    
    return True

# Executar verificação
setup_ok = verify_setup()

🔍 VERIFICAÇÃO DE INTEGRIDADE DO SETUP
✅ Diretório do projeto: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced
✅ Diretório de modelos: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\models
✅ Diretório de outputs: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\outputs
⚠️ Dataset não encontrado: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\data\Task02_Heart
   (Certifique-se de que o dataset está no local correto)
⚠️ GPU não disponível - usando CPU
✅ Seeds configurados: 42
2025-06-18 11:24:08,359 - INFO - Setup verification completed
2025-06-18 11:24:08,362 - INFO - Project path: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\H

In [17]:
# =============================================================================
# 💾 SALVAR CONFIGURAÇÕES
# =============================================================================

def save_config():
    """Salva configurações em arquivo JSON para reutilização"""
    
    config_dict = {
        'project_info': {
            'name': 'Heart Segmentation Advanced',
            'version': '1.0.0',
            'created': datetime.now().isoformat(),
            'environment': 'colab' if IN_COLAB else 'local'
        },
        'paths': {
            'project': config.PROJECT_PATH,
            'dataset': config.DATASET_PATH,
            'models': config.MODELS_PATH,
            'outputs': config.OUTPUTS_PATH
        },
        'data_config': {
            'image_size': config.IMAGE_SIZE,
            'num_classes': config.NUM_CLASSES,
            'class_names': config.CLASS_NAMES,
            'validation_split': config.VALIDATION_SPLIT,
            'test_split': config.TEST_SPLIT
        },
        'training_config': {
            'batch_size': config.BATCH_SIZE,
            'initial_epochs': config.INITIAL_EPOCHS,
            'learning_rate': config.LEARNING_RATE,
            'early_stopping_patience': config.EARLY_STOPPING_PATIENCE
        },
        'model_config': {
            'default_backbone': config.DEFAULT_BACKBONE,
            'backbone_options': config.BACKBONE_OPTIONS
        },
        'augmentation_config': config.AUGMENTATION_PARAMS,
        'loss_config': config.LOSS_WEIGHTS,
        'random_seed': config.RANDOM_SEED
    }
    
    config_file = os.path.join(config.PROJECT_PATH, 'project_config.json')
    
    with open(config_file, 'w') as f:
        json.dump(config_dict, f, indent=2)
    
    print(f"💾 Configurações salvas em: {config_file}")
    logger.info(f"Configuration saved to: {config_file}")

# Salvar configurações
save_config()

💾 Configurações salvas em: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\project_config.json
2025-06-18 11:24:14,630 - INFO - Configuration saved to: c:\Users\leonardo.costa\OneDrive - Lightera, LLC\Documentos\GitHub\pratica-aprendizado-de-maquina\Heart_Segmentation_Advanced\project_config.json


---

## 🎉 Setup Concluído!

O ambiente está configurado e pronto para uso. Próximos passos:

1. **📊 Data Analysis**: Execute `01_Data_Analysis_and_Preprocessing.ipynb`
2. **🔄 Data Augmentation**: Execute `02_Data_Augmentation.ipynb`  
3. **🏗️ Model Architecture**: Execute `03_Model_Architecture.ipynb`

### 📋 Resumo das Configurações

- **Ambiente**: Google Colab ✅
- **GPU**: Disponível e configurada ✅
- **Seeds**: Fixados para reprodutibilidade ✅
- **Paths**: Configurados corretamente ✅
- **Logging**: Sistema ativo ✅
- **Visualização**: Configurada ✅

### 🔧 Configurações Principais

- **Image Size**: 128x128
- **Classes**: 3 (Background, Left Ventricle, Myocardium)
- **Batch Size**: 8
- **Learning Rate**: 0.001
- **Random Seed**: 42

---

**✨ Pronto para começar a análise de dados!**